In [ ]:
pip install pgmpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
model = BayesianNetwork([
    ('Temperature', 'WeatherCondition'),
    ('Humidity', 'WeatherCondition'),
    ('WindSpeed', 'WeatherCondition'),
    ('Pressure', 'WeatherCondition')
])

In [ ]:
cpd_temperature = TabularCPD(variable='Temperature', variable_card=3, values=[[0.3], [0.5], [0.2]])  # Low, Medium, High
cpd_humidity = TabularCPD(variable='Humidity', variable_card=3, values=[[0.4], [0.4], [0.2]])  # Low, Medium, High
cpd_wind_speed = TabularCPD(variable='WindSpeed', variable_card=2, values=[[0.6], [0.4]])  # Low, High
cpd_pressure = TabularCPD(variable='Pressure', variable_card=2, values=[[0.7], [0.3]])  # Low, High


In [ ]:
cpd_weather_condition = TabularCPD(
    variable='WeatherCondition', variable_card=3,
    values=[
        [0.6, 0.5, 0.4, 0.2, 0.1, 0.05, 0.02, 0.01, 0.2, 0.3, 0.4, 0.7,
         0.6, 0.8, 0.3, 0.2, 0.4, 0.1, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2,
         0.1, 0.05, 0.02, 0.01, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.85],  # Rain
        [0.3, 0.4, 0.4, 0.6, 0.8, 0.9, 0.7, 0.8, 0.3, 0.4, 0.3, 0.2,
         0.2, 0.1, 0.5, 0.6, 0.3, 0.7, 0.6, 0.7, 0.5, 0.6, 0.7, 0.8,
         0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.3, 0.4, 0.5, 0.6],  # Sunny
        [0.1, 0.1, 0.2, 0.2, 0.1, 0.05, 0.1, 0.1, 0.5, 0.3, 0.3, 0.1,
         0.2, 0.1, 0.2, 0.2, 0.3, 0.2, 0.3, 0.2, 0.4, 0.2, 0.1, 0.1,
         0.1, 0.15, 0.2, 0.3, 0.2, 0.1, 0.1, 0.2, 0.1, 0.15, 0.05, 0.1]   # Cloudy
    ],
    evidence=['Temperature', 'Humidity', 'WindSpeed', 'Pressure'],
    evidence_card=[3, 3, 2, 2]
)


In [ ]:
num_columns = len(cpd_weather_condition.values[0])  # Get number of columns
for i in range(num_columns):
    column_sum = sum(cpd_weather_condition.values[j][i] for j in range(len(cpd_weather_condition.values)))
    for j in range(len(cpd_weather_condition.values)):
        cpd_weather_condition.values[j][i] /= column_sum

In [ ]:
model.add_cpds(cpd_temperature, cpd_humidity, cpd_wind_speed, cpd_pressure, cpd_weather_condition)

In [ ]:
try:
    if model.check_model():
        print("Model is valid.")
    else:
        print("Model is invalid.")
except ValueError as e:
    print(f"Error in model: {e}")

Model is valid.


In [ ]:
inference = VariableElimination(model)

In [ ]:
query_result = inference.query(
    variables=['WeatherCondition'],
    evidence={
        'Temperature': 1,  # Medium temperature
        'Humidity': 0,     # Low humidity
        'WindSpeed': 1,    # High wind speed
        'Pressure': 1      # High pressure
    }
)
print(query_result)

+---------------------+-------------------------+
| WeatherCondition    |   phi(WeatherCondition) |
+=====================+=========================+
| WeatherCondition(0) |                  0.2000 |
+---------------------+-------------------------+
| WeatherCondition(1) |                  0.6000 |
+---------------------+-------------------------+
| WeatherCondition(2) |                  0.2000 |
+---------------------+-------------------------+
